In [1]:
import os, torch
import numpy as np
import pandas as pd
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from dataPre import *

In [2]:
path_fn = "../../data/pathway_gobp.csv"
gene_fn = "../../data/gene.average.beta.by.intensity.csv"
meth_fn = "../../data/beta.by.intensity.csv"
label_fn = "../../data/label.csv"
pathway_info, expression, methylation, status, features, features_meth = preprocessing2(path_fn, gene_fn, meth_fn, label_fn)

>> Pathway Data : ../../data/pathway_gobp.csv
pathway matrix shape :  torch.Size([5456, 24862])
num_pathway :  5456
>> Methylation Gene-level Data: ../../data/gene.average.beta.by.intensity.csv
sample_dim :  100
input_size (number of genes):  24862
>> Methylation beta Data: ../../data/beta.by.intensity.csv
sample_dim :  100 100
input_size (number of genes, methylation sites):  24862 500257
patient list :  ['GSM7696105', 'GSM7696106', 'GSM7696107', 'GSM7696108', 'GSM7696109', 'GSM7696110']
feature list :  ['DDX11L1', 'LOC101928626', 'FAM87B', 'LINC01128', 'LOC643837', 'FAM41C'] ['cg26928153', 'cg16269199', 'cg26679879', 'cg22519184', 'cg09499020', 'cg15979415']


In [3]:
x_data = np.concatenate((expression,methylation),axis=1)

In [4]:
trainArgs = {}
trainArgs['x_data'] = x_data
trainArgs['y_data'] = status
trainArgs['pathway_info'] = pathway_info
trainArgs['features'] = features
trainArgs['num_fc_list'] = [32]
trainArgs['lr_list'] = [0.001]
#trainArgs['num_fc_list'] = [32]
#trainArgs['lr_list'] = [0.0001]
trainArgs['device'] = '0'
trainArgs['seed'] = 0
trainArgs['filename'] = 'result.csv'

In [5]:
from train import *

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [6]:
train = train_kfold3(trainArgs)
result = train.kfold()

****************************************************************************
Fold 1 / 5
****************************************************************************
MI feature selection...
Validation AUC increased (0.000000 --> 0.500000).  Saving model ...
Validation AUC increased (0.500000 --> 0.550000).  Saving model ...
Validation AUC increased (0.550000 --> 0.550000).  Saving model ...
Validation AUC increased (0.550000 --> 0.650000).  Saving model ...
****************************************************************************
Fold 2 / 5
****************************************************************************
MI feature selection...
Validation AUC increased (0.000000 --> 0.600000).  Saving model ...
****************************************************************************
Fold 3 / 5
****************************************************************************
MI feature selection...
Validation AUC increased (0.000000 --> 0.400000).  Saving model ...
Validation AUC increased 

In [7]:
result

,hyperparam,Fold,Valid_AUC,Valid_Precision,Valid_Recall,Valid_F1,Test_AUC,Test_Precision,Test_Recall,Test_F1,Test_PrAUC
0,lr:0.001 / num_fc:32,0,0.65,0.750000,0.75,0.750000,0.81,0.714286,1.0,0.833333,0.823755
1,lr:0.001 / num_fc:32,1,0.60,0.571429,1.00,0.727273,0.53,0.529412,0.9,0.666667,0.564010
2,lr:0.001 / num_fc:32,2,0.65,0.750000,0.75,0.750000,0.74,0.800000,0.8,0.800000,0.807148
3,lr:0.001 / num_fc:32,3,0.60,0.571429,1.00,0.727273,0.50,0.562500,0.9,0.692308,0.601430
4,lr:0.001 / num_fc:32,4,0.90,0.800000,1.00,0.888889,0.45,0.526316,1.0,0.689655,0.491027


In [8]:
result.to_csv("./result_GOBP_new2.csv")